In [1]:
import warnings
warnings.filterwarnings("ignore")
from utils.io_utils import *
from utils.opensim_kinematics_utils import *
from utils.data_processing_utils import *
from utils.analysis_utils import *

In [ ]:
# define variables 

DATA_FOLDER = "S001"
MARKERSET_FILE = "example_markerset_files/HMB2_MODEL_markers.xml"
# list of the names of the markers in ORIGINAL_MARKERSET_FILE
MARKERS = [
    "LASIS", "RASIS", "LPSIS", "RPSIS", "LLTHI", "LLEK", "LMEK", "LLSHA", "LLM",
    "LMM", "LHEE", "LMT2", "LMT5", "RLTHI", "RLEK", "RMEK", "RLSHA", "RLM", "RMM",
    "RHEE", "RMT2", "RMT5", "C7", "T10", "XIPH", "JN"
    ]
TEMPLATE_SCALING_SETUP_FILE = "template_scaling_setup.xml"

N_DAYS = 15
FILES_TO_CORRECT = "Baseline" # we assume that the files are named "BaselineTx.c3d" with x = 1 - 15
PREFIX = "averaged"
RESULTS_FOLDER = "results_marker_displacement_correction"

In [3]:
os.makedirs(RESULTS_FOLDER, exist_ok=True)  

In [ ]:
# STEP 1: RETRIEVE ALL CALIBRATION .c3d FILES FROM 15 DAYS
original_c3d_calib_files = [os.path.join(DATA_FOLDER, f"CalibrationT{i+1}.c3d") for i in range (N_DAYS)]

# STEP 2: CONVERT ALL CALIBRATION .c3d FILES INTO .trc files
for i in range (N_DAYS):
    original_trc_calib_file = os.path.join(RESULTS_FOLDER, f"CalibrationT{i+1}.trc")
    convert_c3d_to_trc(original_c3d_calib_files[i],
                       output_trc=original_trc_calib_file,
                       start_time=0.0,
                       end_time=1.0,
                       include_markers=MARKERS)
    
# STEP 3: AVERAGE ALL CALIBRATION .trc FILES
averaged_trc_calib_file = os.path.join(RESULTS_FOLDER, f"CalibrationAveraged.trc")
load_average_and_save_trc_bis(RESULTS_FOLDER,
                              os.path.join(RESULTS_FOLDER, "CalibrationT1.trc"),
                              averaged_trc_calib_file,
                              MARKERS,
                              startswith='CalibrationT')

# STEP 4: SCALE AVERAGED MODEL WITH AVERAGE .trc FILE
scaling(PREFIX,
        RESULTS_FOLDER,
        MARKERS,
        TEMPLATE_SCALING_SETUP_FILE,
        MARKERSET_FILE,
        trc_calib_file = averaged_trc_calib_file)

path_to_scaled_model = os.path.join(RESULTS_FOLDER, PREFIX + "_scaled_model_markers.osim")


for i in range (N_DAYS):

    # STEP 5: RUN INVERSE KINEMATICS ON ALL 15 DYNAMIC .trc FILES THAT NEED TO BE CORRECTED USING AVERAGED MODEL
    # here we consider that we want to correct all 15 Baseline files
    (start_time, stop_time) = inverse_kinematics(PREFIX,
                   f"{FILES_TO_CORRECT}T{i+1}",
                   MARKERS,
                   MARKERSET_FILE,
                   path_to_scaled_model,
                   RESULTS_FOLDER,
                   c3d_dynamic_file=os.path.join(DATA_FOLDER, f"{FILES_TO_CORRECT}T{i+1}.c3d"))
    path_to_corrected_motion_file = os.path.join(RESULTS_FOLDER, PREFIX+f"_{FILES_TO_CORRECT}T{i+1}_motion.mot")
    # --> results in 15 corrected .mot files

    # STEP 6: RUN POINT KINEMATICS TO GET CORRECTED MARKER COORDINATES FROM 15 DYNAMIC FILES USING AVERAGED MODEL
    point_kinematics ("corrected",
                  f"{FILES_TO_CORRECT}T{i+1}",
                  MARKERSET_FILE,
                  RESULTS_FOLDER,
                  path_to_scaled_model,
                  path_to_corrected_motion_file,
                  start_time=start_time,
                  end_time=stop_time)
    path_to_corrected_trc_file = os.path.join(RESULTS_FOLDER, f"corrected_{FILES_TO_CORRECT}T{i+1}.trc")
    # --> results in 15 corrected .trc files
    
    # STEP 7: CONVERT ALL CORRECTED .trc files into CORRECTED .c3d files
    convert_trc_to_c3d(path_to_corrected_trc_file, source_c3d_path=os.path.join(DATA_FOLDER, f"{FILES_TO_CORRECT}T{i+1}.c3d"))
    # --> results in 15 corrected .c3d files

759
0 100
✅ Conversion terminée : results_marker_displacement_correction\CalibrationT1.trc
557
0 100
✅ Conversion terminée : results_marker_displacement_correction\CalibrationT2.trc
602
0 100
✅ Conversion terminée : results_marker_displacement_correction\CalibrationT3.trc
590
0 100
✅ Conversion terminée : results_marker_displacement_correction\CalibrationT4.trc
464
0 100
✅ Conversion terminée : results_marker_displacement_correction\CalibrationT5.trc
742
0 100
✅ Conversion terminée : results_marker_displacement_correction\CalibrationT6.trc
408
0 100
✅ Conversion terminée : results_marker_displacement_correction\CalibrationT7.trc
679
0 100
✅ Conversion terminée : results_marker_displacement_correction\CalibrationT8.trc
481
0 100
✅ Conversion terminée : results_marker_displacement_correction\CalibrationT9.trc
717
0 100
✅ Conversion terminée : results_marker_displacement_correction\CalibrationT10.trc
426
0 100
✅ Conversion terminée : results_marker_displacement_correction\CalibrationT11.t